<a href="https://colab.research.google.com/github/Nnnnishimura/-/blob/main/daikon(%E9%80%9A%E3%81%97%2C1280_720%E7%94%BB%E8%B3%AA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-1o3opdp4
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-1o3opdp4


In [2]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
# import some common libraries
import glob
import math

In [5]:
#推論＆マスク画像作成

In [6]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 1クラスのみ
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6

predictor = DefaultPredictor(cfg)

[10/03 07:13:41 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                  | Names in Checkpoint                                                                                  | Shapes                                          |
|:------------------------------------------------|:-----------------------------------------------------------------------------------------------------|:------------------------------------------------|
| backbone.bottom_up.res2.0.conv1.*               | backbone.bottom_up.res2.0.conv1.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| backbone.bottom_up.res2.0.conv2.*               | backbone.bottom_up.res2.0.conv2.{norm.bias,norm.running_mean,norm.running_var,norm.weight,weight}    | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| backbone.bottom_up.res2.0.conv3.*               | backbone.bottom_up.res2.0.conv3.{norm.bia

In [7]:
import time

In [164]:
start_time = time.perf_counter()
imgPath = "/content/drive/MyDrive/GX010181_605 (1280-720).png"
im = cv2.imread(imgPath)

outputs = predictor(im)
mask = outputs['instances'].pred_masks
mask_nd = mask.to('cpu').detach().numpy().copy()
mask_reshape = np.reshape(mask_nd,(921600))
mask_img = []
for item in mask_reshape:
  if item == False:
    mask_img.append(0)
  else:
    mask_img.append(255)

mask_img_reshape = np.reshape(mask_img,(720,1280))
mask_img_reshape = mask_img_reshape.astype(np.uint8)
ret, mask_binary = cv2.threshold(mask_img_reshape, 10, 255, cv2.THRESH_BINARY)

In [165]:
#曲り判定

In [166]:

ret,imthresh = cv2.threshold(mask_binary,127,255,cv2.THRESH_BINARY)
im_copy = imthresh.copy()

In [167]:
#大根の輪郭抽出
contours , hierarchy = cv2.findContours(im_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [168]:
#凸包の作成
hull=[cv2.convexHull(c) for c in contours]

In [169]:
#凸包を形成する点群の座標リストの作成
hull = cv2.convexHull(contours[0])
hull_pts = np.array(hull)                                     #この時のhull_ptsは3次元配列。（奥行き、行数、列数）                                       
hull_pts = hull_pts.reshape(hull.shape[0],hull.shape[2])      #hull_ptsを3次元配列から2次元配列に変換。（行数44、列数2）

In [170]:
#接線の導出
previous=[]                 #空のリストを作成
max_hull_dis=0              #max_hull_dis=その時点での最大の距離(接線の候補)
for hull_pt in hull_pts:    #座標リストのインデックスを順番にhull_ptに代入
  if previous == []:        #座標リスト１つ目のインデックス[1054,284]を代入
    previous=hull_pt        #previous=[1054,284]
    initial=hull_pt         #initial=[1054,284]

  else:   #座標リストの２つ目以降のインデックス
    dis = (hull_pt[0]-previous[0])**2 + (hull_pt[1]-previous[1])**2  #１つ前の座標との距離を計算
    if dis > max_hull_dis:                                           #今までで一番長い距離と比較して上回ったら
      max_hull_dis = dis                                             #その距離をmax_hull_disに登録
      line_pt1=previous                                              #接線候補を形成する１つ目の点の座標
      line_pt2=hull_pt                                               #接線候補を形成する２つ目の点の座標
    previous = hull_pt                                               #previousを現座標に置き換え

dis = (previous[0]-initial[0])**2 + (previous[1]-initial[1])**2      #座標リストの最初と最後の点の距離を計算
if dis > max_hull_dis:                                              
  line_pt1=previous
  line_pt2=initial

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """


In [171]:
#大根の輪郭を形成する点群のリスト作成

contour_pts = np.array(contours)      #この時のcontour_ptsは4次元配列。
contour_pts = contour_pts.reshape(contour_pts.shape[1] , contour_pts.shape[3])

In [172]:
for  index, contour_pt  in enumerate(contour_pts):  #index = リストの番号　contour_pt = 実際の座標
  if np.all(contour_pt == line_pt2):
    index_2 = index                                 #index_2 = line_pt2のリストの番号(インデックス)
  elif np.all(contour_pt == line_pt1):
    index_1 = index                                 #index_1 = line_pt1のリストの番号(インデックス)

inside_pts = contour_pts[index_2 :index_1 + 1]      #inside_pts = 内側輪郭を形成する点群のリスト

In [173]:
inside_buffer = np.zeros(im.shape[:2], dtype=np.uint8)   #行数1440、列数1920で数値がすべて0の2次元配列を作成   #この時のinside_ptsは行数287、列数２の2次元配列                                 
inside_pts = np.reshape(inside_pts , (inside_pts.shape[0], 1, inside_pts.shape[1]))    #inside_ptsを奥行き287、行数１，列数２の３次元配列に変換

In [174]:
inside_output = cv2.polylines(inside_buffer, [inside_pts], True, 255, 1)   #第1引数：画像　第2引数：点群の座標　第3引数：True＝終点と始点を結ぶ　第4引数：線の色　第5引数：線の太さ

In [175]:
#line_pt1のX座標、Y座標のデータ型をそれぞれnumpy.int64からintに変換
line_pt10=line_pt1[0].item()
line_pt11=line_pt1[1].item()

In [176]:
#画像の回転
if line_pt1[0] != line_pt2[0]:                                        #line_pt1[0],line_pt[0] = 接線の両端のX座標
  slope = -(line_pt1[1] - line_pt2[1])/(line_pt1[0] - line_pt2[0])    #接線の傾きにマイナスをつける　　左上が原点となるため
  theta = math.degrees(math.atan(slope))                              #math.atan() : tanの逆関数を得る　math.degrees() : ラジアンを度数に変換する
  if theta < 0:
    yaw = -90 - theta
  else:
    yaw = 90 - theta                                                  #yaw =　回転させる角度
  mat = cv2.getRotationMatrix2D((line_pt10,line_pt11),yaw,1)          #回転の変換行列を作成　 第1引数：回転の原点座標　第2引数：回転角度　第3引数：拡大・縮小倍率　　　　
  affine_output = cv2.warpAffine(inside_output,mat,(1920,1440))       #画像の回転(アフィン変換)　第1引数：元画像　第2引数：変換行列　第3引数：出力画像のサイズ

In [177]:
dis_pts = list(zip(*np.where(affine_output >= 1))) #np.where(affine_output >= 1) : 画像内で色が1以上（=白線）の座標の高さ(Y座標)と幅(X座標)をそれぞれリストにして作成
                                                   #zip(*np.where(affine_output >= 1) : 上の高さと幅の座標リストをインデックス順に組み合わせて座標リストを作成(タプル)

In [178]:
dis_pts = np.array(dis_pts)   #白線を形成する点群の座標をリスト化

In [179]:
dis_x_max = dis_pts.max(axis=0)[1]         #白線を形成する点群の中でX座標値が最も大きい点(１番右に位置する点)のX座標①を抽出
dis_x_min = dis_pts.min(axis=0)[1]         #白線を形成する点群の中でX座標値が最も小さい点(１番左に位置する点)のX座標②を抽出
dis_max = abs(dis_x_max - line_pt1[0])     #接線の端の点(回転軸)のX座標と①の距離を計算
dis_min = abs(dis_x_min - line_pt1[0])     #接線の端の点(回転軸)のX座標と②の距離を計算
if dis_max >= dis_min:
  dis_x = dis_x_max                                    
  ans_pts = list(zip(*np.where(dis_pts == dis_x)))
  distance = dis_max

else:
  dis_x = dis_x_min
  ans_pts = list(zip(*np.where(dis_pts == dis_x)))
  distance = dis_min                      #distance = ピクセル単位での曲がり距離

In [180]:
if distance <= 26.11:
    print("distance:"+str(distance)+"[pixel](excellent)")
elif distance > 26.11 and distance <= 52.22:
    cv2.putText(im, "distance:"+str(distance)+"[pixel](good)",(50, 30), cv2.FONT_HERSHEY_PLAIN, 1.5, 255, 2)
    cv2_imshow(im)
else:
    print("distance:"+str(distance)+"[pixel](cut)")
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(elapsed_time)

distance:61[pixel](cut)
2.021324760000425
